In [1]:
import render
import rooms.dataset
import torch
import torch.nn as nn
import matplotlib.pyplot as plt

In [2]:
torch.set_default_dtype(torch.float32)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

/home/gzanin/.local/lib/python3.11/site-packages/torch/cuda/__init__.py:118: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0


In [3]:
D_0 = rooms.dataset.dataLoader("classroomBase")

In [4]:
L_0 = render.get_listener(source_xyz=D_0.speaker_xyz, listener_xyz = D_0.xyzs[0], surfaces=D_0.all_surfaces, 
                                                speed_of_sound=D_0.speed_of_sound,
                                                parallel_surface_pairs=D_0.parallel_surface_pairs,
                                                max_order=D_0.max_order, max_axial_order=D_0.max_axial_order )

Considered Paths:	59049
Total Considered Paths, after Axial:	59079
Valid Paths:	851


In [5]:
R_0 = render.Renderer(n_surfaces=len(D_0.all_surfaces))

In [6]:
# Use more GPUs if available
if torch.cuda.device_count() > 1:
    print(f"Using {torch.cuda.device_count()} GPUs")
    R_0 = nn.DataParallel(R_0).module

In [7]:
pt_file = torch.load('./models/classroomBase/weights.pt', map_location=device)
R_0.energy_vector = nn.Parameter(pt_file['model_state_dict']['energy_vector'])
R_0.source_response = nn.Parameter(pt_file['model_state_dict']['source_response'])
R_0.directivity_sphere = nn.Parameter(pt_file['model_state_dict']['directivity_sphere'])
R_0.decay = nn.Parameter(pt_file['model_state_dict']['decay'])
R_0.RIR_residual = nn.Parameter(pt_file['model_state_dict']['RIR_residual'])
R_0.spline_values = nn.Parameter(pt_file['model_state_dict']['spline_values'])

R_0.bp_ord_cut_freqs.to(device) 

Parameter containing:
tensor([  70.,  400.,  800., 1000., 1300., 2000.], requires_grad=True)

In [8]:
resp = R_0.render_RIR_directional(L_0)
resp

time to initializie freq resp =  0.09229874610900879
time to introduce delays =  3.769803524017334
time to compute az and ele and set up for the beampatterns:  0.002107858657836914


In [ ]:
# Plot for direction 1
plt.plot(resp[1]['t_response'].detach().cpu())
plt.title("RIR in direction 1")
plt.xlabel("Sample")
plt.ylabel("Value")
plt.grid(True)
plt.show()

In [ ]:
# Plot of the beginning for direction 1
plt.plot(resp[1]['t_response'][:1500].detach().cpu())
plt.title("Initial part of RIR in direction 1")
plt.xlabel("Sample")
plt.ylabel("Value")
plt.grid(True)
plt.show()

In [ ]:
resp0 = R_0.render_RIR(L_0)
resp0

In [ ]:
# Plot of the omnidirectional RIR
plt.plot(resp0.detach().cpu())
plt.title("Omnidirectionalo RIR")
plt.xlabel("Sample")
plt.ylabel("Value")
plt.grid(True)
plt.show()